# Model fine-tuning
Fine-tuning of a GPT-2 small Spanish model to generate lullabies or "nanas".

We have used this notebook as reference
https://colab.research.google.com/drive/13dZVYEOMhXhkXWfvSMVM1TTtUDrT6Aeh?usp=sharing#scrollTo=NKGBoVwuhM4H but with an spanish model ([datificate/gpt2-small-spanish](https://huggingface.co/datificate/gpt2-small-spanish)).

## Setup
Install dependencies and import libraries needed.

In [ ]:
# Check Transformers Library from HuggingFace
! pip install transformers

     |████████████████████████████████| 2.1MB 16.4MB/s 
     |████████████████████████████████| 901kB 53.4MB/s 
     |████████████████████████████████| 3.3MB 55.3MB/s 


In [ ]:
import torch
import random
import numpy as np
import datetime
import time

from transformers import AutoTokenizer, AutoModelWithLMHead, AdamW, get_linear_schedule_with_warmup

## GPT-2 (spanish) fine-tuning

### Data loading

In [ ]:
data_list = [
"""Duérmete, niñito mío,
que tu madre no está en casa;
que se la llevó la Virgen
de compañera a su casa.""",
"""Duérmete, niño chico,
duérmete, mi bien;
que aquí está la cunita
que te ha de mecer.""",
"""Duérmete, vida mía,
duerme sin pena,
porque al pie de la cuna
tu madre vela.""",
"""Duerme, niño mío,
que tengo que hacer,
me han traído el trigo
y está por moler.""",
"""Duérmete, niño de cuna,
duérmete, niño de amor,
que a los pies tienes la luna
y a la cabecera el sol.""",
"""Duérmete, mi niño,
que viene el coco
y se lleva a los niños
que duermen poco.""",
"""Duérmete, niño,
duérmete ya
que viene el coco
y te comerá.""",
"""Este galapaguito no tiene mare.
lo parió una gitana,
lo echó a la calle.
Este niño chiquito
no tiene cuna.
Su padre es carpintero
y le hará una cuna.
Y le hace una de caramelo
para que cuando despierte
se chupe el dedo""",
"""Al sueño le crecen		
cabellos de yerba.		
Al sueño le nacen azules gacelas,		
que muerden los prados,		
que triscan las eras;		
que pacen las noches		
sin que el sueño pueda		
cortarse sus ramas	
de verdes almendras."""
"""Al sueño le llaman		
y el sueño contesta,		
con sus ojos claros		
y su boca lenta,		
que dice palabras		
que el sueño se inventa.		
Duérmete, mi vida,		
niña de la tierra:		
que el sueño te canta		
para que te duermas.""",
"""Duérmete, niño chico,		
duérmete, mi bien;		
que aquí está la cunita		
que te ha de mecer.""",
"""En los brazos te tengo		
y considero		
qué será de ti, niño,		
si yo me muero.""",
"""Duerme, niño chiquito,		
duérmete y calla;		
no le des a tu madre		
tanta batalla.""",
"""Este niño tiene sueño,		
no tiene cama ni cuna.		
A su padre carpintero		
le diremos le haga una .""",
"""Si este niño se durmiera,		
yo le diera medio real,		
para que se comprara		
un pedacito de pan.""",
"""Duérmete, niño,		
que ahí viene el coco,		
y se lleva a los niños		
que duermen poco.""",
"""Duérmete mi niño		
y duérmete ya,		
porque viene el coco		
y te comerá.""",
"""Con decirle a mi niño		
que viene el coco,		
le va perdiendo el miedo		
poquito a poco.""",
"""Las mujeres de la sierra,		
para dormir a sus niños,		
en vez de llamar al coco		
les cantan un fandanguillo.""",
"""Y arrorró, canelica,		
que viene el coco		
y se lleva a los nenes		
que duermen poco.		
Mi chico se va a dormir		
porque tiene mucho sueño,		
y por cabecera tiene		
a la Virgen del Remedio.""",
"""Original persona pequeñita		
que al contrario de todos		
no has nacido.		
Vívete, niño, vívete,		
que viene el Coco		
y se lleva a los niños		
que duermen poco."""		
"""Late un momento rey		
-la madre dice-,		
deja que me dé tiempo		
a que te bautice.		
Te iba a poner Tomás,		
y ya te vas.		
Para qué habrás venido		
sin más ni más.	
Qué frío tienes, hijo,		
sin un temblor,		
creo que dentro estabas		
mucho mejor."""
"""-En el lago de llanto		
de tu madre		
jugabas en la orilla…
- Que el demonio se lleve
tu canastilla
-Tiene ojos de listo,
es un pequeño sabio.
Y otra vecina dijo:	
-De buena se ha librado."""	
"""Pequeño criminal,		
dulce adversario		
-sin nacer ni morir		
a tu madre has matado-,		
mientras tú,		
mi niño diferente,		
ni blanco ni negro,		
mientras tú…		
Échate un sueño largo,		
mi niño azul.""",
"""A la mar, si no duermes,		
que viene el viento.		
Ya en las grutas marinas		
ladran sus perros.		
Si no duermes, al monte.		
Vienen el búho		
y el gavilán del bosque.		
Cuando te duermas:		
al almendro, mi niño,		
y a la estrella de menta.""",
"""Duérmete, niñito,		
duérmete, por Dios,		
si no viene el brujo		
y te va a comer.""",
"""Con un traje rico		
y su hijito feo,		
la loba, la loba,		
vendrá por aquí,		
si esta niña linda		
no quiere dormir.""",
"""Duérmete, guagüita,		
que viene la cierva		
a saltos y brincos		
por entre las piedras.""",
"""Dúermete, niñito,		
cabeza de ayote;		
si no te dormís,		
te come el coyote.""",
"""Duérmete, niño chiquito,		
duérmete y no llores más,		
que se irán los angelitos		
para no verte llorar.""",
"""A los niños buenos		
Dios los bendice;		
a los que son malos		
les da lombrices.""",
"""Duérmete mi negrazo,		
cara de pambazo,		
que si no se duerme		
le doy un trancazo.""",
"""En los brazos te tengo		
y considero		
qué será de ti, niño,		
si yo me muero.""",
"""Duérmete, niño, en la cuna
que viene la reina mora
preguntando por las casas
quién es el niño que llora.""",
"""Duérmete, niño chiquito,
mira que viene la mora
preguntando puerta en puerta
cuál es el niño que llora.""",
"""Vete ya, lobo malo,
que el nene duerme.
Márchate despacito,
no se despierte.""",
"""Milano negro que vuelas
sobre el techo de mi casa.
¡Vete, milano! que al niño
le estoy cantando una nana.""",
"""Angelitos del cielo
venir cantando
y llevarse a este niño,
que está llorando.""",
"""Si este niño no se duerme,
Venga un ángel y lo lleve.
–No vengas, angelito, no,
que este niño se durmió.""",
"""Caballero en la jaca con alas
se vino y le lleva
montado a la grupa,
se vino y le lleva
volando, volando, volando
mi niño… mi prenda.""",
"""Este niño no puede dormir.
El coquito no le deja,
que le tiene agarradito
de los pies a la cabeza.""",
"""Sabes tú, niño,
que quiere el coco:
que tengas miedo,
ni mucho ni poco.""",
"""Duérmete, mi niño,
duérmete sin miedo,
aunque silben los aires,
gruñan los perros.""",
"""Ya no dicen las madres
Que viene el coco;
Que esta voz a los niños
Asusta poco.
Si el caso apura,
Le dicen: Calla, niño,
Que viene el cura""",
"""Las mujeres de la sierra
para dormir los chiquillos
en vez de cantar el coco
le arrean con un ladrillo
y le duermen poco a poco."""
]

In [ ]:
print("Total number of lullabies: ", len(data_list))

Total number of lullabies:  41


### GPT-2 small spanish model loading

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("datificate/gpt2-small-spanish")

model = AutoModelWithLMHead.from_pretrained("datificate/gpt2-small-spanish")

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:762: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [ ]:
print("The max model length is {} for this model, although the actual embedding size for GPT small is 768".format(tokenizer.model_max_length))
print("The beginning of sequence token {} token has the id {}".format(tokenizer.convert_ids_to_tokens(tokenizer.bos_token_id), tokenizer.bos_token_id))
print("The end of sequence token {} has the id {}".format(tokenizer.convert_ids_to_tokens(tokenizer.eos_token_id), tokenizer.eos_token_id))
print("The padding token {} has the id {}".format(tokenizer.convert_ids_to_tokens(tokenizer.pad_token_id), tokenizer.pad_token_id))

The max model length is 1000000000000000019884624838656 for this model, although the actual embedding size for GPT small is 768
The beginning of sequence token <|endoftext|> token has the id 0
The end of sequence token <|endoftext|> has the id 0
The padding token <|endoftext|> has the id 0


In [ ]:
batch_size = 1

In [ ]:
class GPT2Dataset(torch.utils.data.Dataset):

  def __init__(self, txt_list, tokenizer, gpt2_type="gpt2", max_length=768):

    self.tokenizer = tokenizer
    self.input_ids = []
    self.attn_masks = []

    for txt in txt_list:

      encodings_dict = tokenizer('<|endoftext|>' + txt + '<|endoftext|>', truncation=True, max_length=max_length, padding="max_length")

      self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
      self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))
    
  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.attn_masks[idx] 

In [ ]:
dataset = GPT2Dataset(data_list, tokenizer, max_length=768)

# Split into training and validation sets
train_size = int(0.95 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

   38 training samples
    3 validation samples


In [ ]:
train_dataloader = torch.utils.data.DataLoader(
            train_dataset,  # The training samples.
            sampler = torch.utils.data.RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = torch.utils.data.DataLoader(
            val_dataset, # The validation samples.
            sampler = torch.utils.data.SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

### Training

In [ ]:
! nvidia-smi

Sat May  8 03:45:44 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    28W /  70W |   5952MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Select device
if (torch.cuda.is_available()):
    device = torch.device("cuda")
    print("Running on GPU")
else:
    device = torch.device("cpu")
    print("Running on CPU")


Running on GPU


In [ ]:
# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
# some parameters that work reasonably well
epochs = 40
learning_rate = 5e-4
warmup_steps = 2
epsilon = 1e-8

# this produces sample output every 10 steps
sample_every = 10

In [ ]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
optimizer = AdamW(model.parameters(),
                  lr = learning_rate,
                  eps = epsilon
                )

In [ ]:
# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
# This changes the learning rate as the training loop progresses
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = warmup_steps, 
                                            num_training_steps = total_steps)

In [ ]:
def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))

In [ ]:
total_t0 = time.time()

training_stats = []

model = model.to(device)

for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()

    total_train_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):

        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        model.zero_grad()        

        outputs = model(  b_input_ids,
                          labels=b_labels, 
                          attention_mask = b_masks,
                          token_type_ids=None
                        )

        loss = outputs[0]  

        batch_loss = loss.item()
        total_train_loss += batch_loss

        # Get sample every x batches.
        if step % sample_every == 0 and not step == 0:

            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}. Loss: {:>5,}.   Elapsed: {:}.'.format(step, len(train_dataloader), batch_loss, elapsed))

            model.eval()

            sample_outputs = model.generate(
                                    bos_token_id=random.randint(1,30000),
                                    do_sample=True,   
                                    top_k=50, 
                                    max_length = 30,
                                    top_p=0.95, 
                                    num_return_sequences=1
                                )
            for i, sample_output in enumerate(sample_outputs):
                  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))
            
            model.train()

        loss.backward()

        optimizer.step()

        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)       
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()

    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)
        
        with torch.no_grad():        

            outputs  = model(b_input_ids, 
#                            token_type_ids=None, 
                             attention_mask = b_masks,
                            labels=b_labels)
          
            loss = outputs[0]  
            
        batch_loss = loss.item()
        total_eval_loss += batch_loss        

    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    validation_time = format_time(time.time() - t0)    

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 40 ========
Training...


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch    10  of     38. Loss: 0.7960211634635925.   Elapsed: 0:00:03.
0:  máximas La


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch    20  of     38. Loss: 1.0165116786956787.   Elapsed: 0:00:06.
0: ensión,


...

  Batch    10  of     38. Loss: 0.029529031366109848.   Elapsed: 0:00:03.
0:  ClaseDuérmete, niño chico,
duérmete ya
que aquí está la cunita
que te ha de mecer.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch    20  of     38. Loss: 0.006891859695315361.   Elapsed: 0:00:06.
0:  crédiDuérmete, vida mía,
duerme sin pena,
porque al pie de la cuna
tu madre vela.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch    30  of     38. Loss: 0.01500753778964281.   Elapsed: 0:00:10.
0:  congreCon decirle a mi niño	
que viene el coco	
y se lleva a los niños		
que duermen poco.

  Average training loss: 0.01
  Training epoch took: 0:00:12

Running Validation...
  Validation Loss: 0.19
  Validation took: 0:00:00

Training complete!
Total training took 0:08:19 (h:mm:ss)


### Save fine-tuned model

Mount google drive and save the model in a specific folder.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%cd '/content/gdrive/MyDrive/AI song contest/'

/content/gdrive/MyDrive/AI song contest


In [ ]:
model.eval() # ensure the model is 'frozen'
torch.save(model.state_dict(), 'Models/model_nana_v3.pt')